# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.dropna()
cities_df.head()

,City_ID,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ossora,59.24,163.07,30.40,96,100,25.03,RU,1579918884
1,1,busselton,-33.65,115.33,84.99,30,0,5.37,AU,1579919047
2,2,longyearbyen,78.22,15.64,1.40,49,20,18.34,SJ,1579919047
3,3,vaini,-21.20,-175.20,84.20,74,40,13.87,TO,1579918997
4,4,atuona,-9.80,-139.03,80.96,77,0,12.80,PF,1579918892


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat", "lng"]]
humidity = cities_df["Humidity"].astype(float)

In [4]:
Heatmap = gmaps.figure(center = [0,0] ,zoom_level = 2)
 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
Heatmap.add_layer(heat_layer)
Heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
temp_file=cities_df.loc[(cities_df['Max Temp']>=70)&(cities_df['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City_ID,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
59,59,rocha,-34.48,-54.33,70.83,49,0,7.90,UY,1579919059
65,65,mercedes,-34.65,-59.43,79.00,74,0,1.99,AR,1579919059
82,82,salto,-31.38,-57.97,73.00,89,0,4.45,UY,1579918847
94,94,les cayes,18.20,-73.75,78.55,68,0,7.61,HT,1579919062
166,166,cidreira,-30.18,-50.21,72.99,70,0,2.13,BR,1579919081


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(base_url, params=params)

    hotel_name = hotel_name.json()
    
    # Since some data may be missing adding try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

,City_ID,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,59,rocha,-34.48,-54.33,70.83,49,0,7.90,UY,1579919059,Bahia Vik Jose Ignacio
1,65,mercedes,-34.65,-59.43,79.00,74,0,1.99,AR,1579919059,Hotel Sofitel La Reserva Cardales
2,82,salto,-31.38,-57.97,73.00,89,0,4.45,UY,1579918847,Los Naranjos Resort & Spa
3,94,les cayes,18.20,-73.75,78.55,68,0,7.61,HT,1579919062,Villa Mimosa
4,166,cidreira,-30.18,-50.21,72.99,70,0,2.13,BR,1579919081,Pousada Encanto da Praia
5,222,zaraza,9.35,-65.32,73.49,83,0,9.55,VE,1579919093,Hotel Roraima Suites
6,247,port lincoln,-34.73,135.87,75.43,48,0,3.85,AU,1579919096,Marina Hotel
7,446,yacuiba,-22.04,-63.68,72.03,54,0,4.14,BO,1579919141,Hostal Bolivia
8,462,paso de carrasco,-34.86,-56.05,73.40,60,0,6.93,UY,1579919149,Hotel Sofitel Montevideo Casino Carrasco and Spa
9,518,wawa,9.90,4.42,72.01,26,0,6.93,NG,1579919160,Kainji Parkway Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

Heatmap.add_layer(markers)

# Display Map
Heatmap

Figure(layout=FigureLayout(height='420px'))